In [1]:
import pandas as pd
import numpy as np

In [597]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [155]:
test

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...
...,...,...
26255,169728316,Выиграй коллекционный пазл по Wortokenoid of W...
26256,279369911,Волейбол от первого лица Егора Пупынина переко...
26257,600699419,Вы были когда нибудь на свидании где вам задав...
26258,560223506,ТОП 20 самых эффективных общефизических упражн...


In [116]:
train

,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...
...,...,...,...
38735,910636962,autosport,8 битная буря снова накрыла пикселями автомоби...
38736,669736851,autosport,Ира Сидоркова объясняет как сказалась на ее ма...
38737,558919241,tennis,24 я ракетка мира хорват Марин Чилич обыграл и...
38738,776944963,volleyball,Стал известен календарь мужской сборной России...


## предобработка

In [117]:
import re
import numpy as np
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from multiprocessing import Pool
from pymystem3 import Mystem
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/max/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [118]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [119]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = stopwords.words('russian')

# mystem = Mystem()
# def lemmatize(words, lemmer = mystem, stopwords = stopwords_list):
#     lemmas = lemmer.lemmatize(' '.join(words))
#     return [w for w in lemmas if not w in stopwords 
#             and w.isalpha()]

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

In [120]:
train.text.iloc[1]

'Возвращение в подземелье Треша 33 Эйфория тупости и жадности зашкаливает 33 В этот раз мы посетим Ростовскую область. Всю. И знаете это пока что единственный выпуск в котором не будет. . . А впрочем сами все увидите 33 Ссылка на видео в источнике'

In [121]:
print(preprocess(train.text.iloc[1]))

['возвращение', 'подземелье', 'треш', 'эйфория', 'тупость', 'жадность', 'зашкаливать', 'посетить', 'ростовский', 'область', 'весь', 'знать', 'это', 'пока', 'единственный', 'выпуск', 'который', 'всё', 'увидеть', 'ссылка', 'видео', 'источник']


In [598]:
train.category.value_counts()

autosport       3160
extreme         3110
martial_arts    3050
motosport       3030
boardgames      3020
tennis          3000
esport          2990
athletics       2970
hockey          2950
volleyball      2950
football        2860
basketball      2850
winter_sport    2800
Name: category, dtype: int64

In [599]:
preprocessed_text = list(tqdm(map(preprocess, train['text']), total=len(train)))

  0%|          | 0/38740 [00:00<?, ?it/s]

In [156]:
preprocessed_text_test = list(tqdm(map(preprocess, test['text']), total=len(test)))

  0%|          | 0/26260 [00:00<?, ?it/s]

In [603]:
train['text'] = preprocessed_text
train.sample(3)

,oid,category,text
19814,679447598,winter_sport,"[слуцкий, эмоция, трусов, увидеть, олимпиада, ..."
32272,58456467,basketball,"[сайт, tokentokenoid, платный, подписка, добав..."
12648,866431267,winter_sport,"[очень, хотеться, валиев, допустить, соревнова..."


In [244]:
test['text'] = preprocessed_text_test

In [126]:
from gensim.models import Word2Vec, FastText

In [604]:
modelW2V = Word2Vec(sentences=train['text'], vector_size=300, window=5, min_count=3)
# modelFT = FastText(sentences=train['text'], vector_size=300, min_count=3, window=5, workers=8)

In [32]:
# %%time
# w2v_index = annoy.AnnoyIndex(300 ,'angular')
# ft_index = annoy.AnnoyIndex(300 ,'angular')

# index_map = {}
# counter = 0


# for sent in train:
#     n_w2v = 0
#     n_ft = 0
#     index_map[counter] = sent
#     vector_w2v = np.zeros(300)
#     vector_ft = np.zeros(300)
#     for word in sent:
#         if word in modelW2V.wv:
#             vector_w2v += modelW2V.wv[word]
#             n_w2v += 1
#         if word in modelFT.wv:
#             vector_ft += modelFT.wv[word]
#             n_ft += 1
#     if n_w2v > 0:
#         vector_w2v = vector_w2v / n_w2v
#     if n_ft > 0:
#         vector_ft = vector_ft / n_ft
#     w2v_index.add_item(counter, vector_w2v)
#     ft_index.add_item(counter, vector_ft)

#     counter += 1

# w2v_index.build(10)
# ft_index.build(10)

CPU times: user 1.98 ms, sys: 2.69 ms, total: 4.67 ms
Wall time: 6.15 ms


True

In [605]:
def get_vector(sent, model):
    vector = np.zeros(300)
    norm = 0
    for word in sent:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    return vector

In [606]:
w2v_lst = train['text'].apply(lambda x: get_vector(x, modelW2V))

In [133]:
# ftt_lst = train['text'].apply(lambda x: get_vector(x, modelFT))

In [247]:
w2v_lst_test = test['text'].apply(lambda x: get_vector(x, modelW2V))

In [607]:
w_lst = []
for i in range(len(w2v_lst)):
    w_lst.append(list(np.array(w2v_lst)[i]))

In [248]:
w_lst_test = []
for i in range(len(w2v_lst_test)):
    w_lst_test.append(list(np.array(w2v_lst_test)[i]))

In [173]:
# f_lst = []
# for i in range(len(ftt_lst)):
#     f_lst.append(list(np.array(ftt_lst)[i]))

In [608]:
classes = {
    0: -1,
    'athletics': 0,
    'autosport': 1,
    'basketball': 2,
    'boardgames': 3,
    'extreme': 4,
    'football': 5,
    'hockey': 6,
    'martial_arts': 7,
    'motosport': 8,
    'tennis': 9,
    'volleyball': 10,
    'winter_sport': 11
}

In [609]:
train_w2v = pd.DataFrame(w_lst)
# train_ftt = pd.DataFrame(f_lst)

In [558]:
test_w2v = pd.DataFrame(w_lst_test)


In [610]:
train_w2v.shape

(38740, 300)

In [611]:
train_w2v['category'] = train['category']
# train_ftt['category'] = train['category']

In [612]:
train_w2v_min = train_w2v.loc[train_w2v['category'] != 'esport']

In [615]:
train_w2v_min.shape

(35750, 301)

In [332]:
test_w2v['oid'] = test['oid']

In [180]:
for i in classes.keys():
    train_w2v.loc[train_w2v['category']==i, 'category'] = classes[i]
    

In [614]:
for i in classes.keys():
    train_w2v_min.loc[train_w2v_min['category']==i, 'category'] = classes[i]


In [616]:
pd.unique(train_w2v_min['category'])

array([11, 4, 5, 3, 6, 0, 8, 2, 9, 1, 7, 10], dtype=object)

In [588]:
# for i in classes.keys():
#     train_ftt.loc[train_ftt['category']==i, 'category'] = classes[i]

In [617]:
X = train_w2v_min.drop(columns='category')
y = train_w2v_min['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)

In [645]:
X

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.301024,0.021623,0.249025,-0.206230,-0.214803,-0.254629,0.796195,0.432808,0.177280,-0.165865,...,0.354328,0.090016,0.215216,-0.207424,-0.230553,-0.164203,-0.022586,-0.044179,0.373197,0.254151
1,0.021163,0.025198,0.178981,0.108735,0.139897,-0.062942,-0.126535,0.470258,0.177759,0.011005,...,-0.032219,0.158603,0.213771,0.310990,0.515446,0.299836,-0.079033,-0.064240,0.183142,-0.046618
2,0.040905,0.166485,0.021429,0.064506,0.009870,-0.173128,-0.098547,0.630060,0.092194,0.008872,...,0.023663,0.103020,0.224853,0.214985,0.450492,0.464360,-0.025736,-0.260059,0.132263,-0.359151
3,0.007688,0.075425,0.197671,0.338271,0.163249,-0.110499,0.086123,0.409598,0.226518,0.108192,...,-0.058390,0.145729,0.148410,0.021355,0.254888,0.204228,-0.208476,-0.043212,0.203342,0.035738
4,0.036132,0.133871,0.010075,-0.056599,-0.033589,-0.104970,-0.018969,0.712543,0.199854,-0.021556,...,0.284608,0.059379,0.207499,0.186348,0.268998,0.366692,0.031245,-0.099991,0.203589,-0.395702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38735,0.002409,0.145437,0.008523,0.149028,0.066334,-0.115968,-0.132636,0.454721,0.133341,-0.048314,...,-0.130298,0.243696,0.274931,0.280852,0.554414,0.293370,-0.043810,-0.182421,0.114808,-0.202851
38736,0.147472,0.317703,-0.315193,-0.184259,-0.030265,-0.312730,-0.021322,0.592946,0.068569,-0.260362,...,0.018894,0.347562,0.352691,0.297544,0.525008,0.584124,-0.020625,0.003995,0.007346,-0.540119
38737,0.050912,0.446290,-0.146800,-0.037136,-0.015129,-0.398997,0.187198,0.791379,0.020784,-0.477984,...,0.165569,0.248366,0.186140,-0.321866,-0.122842,0.357923,0.069378,-0.065721,0.496530,-0.518595
38738,-0.350177,0.234302,-0.192607,-0.501729,-0.695365,-0.157569,-0.143657,0.933690,0.631427,-0.520937,...,0.302884,0.065593,0.380396,-0.035699,0.549061,0.104659,0.047098,-0.091604,0.449231,-0.518577


In [619]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [620]:
clf = LogisticRegression()
clf.fit(X_train, list(y_train))

/Users/max/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [722]:
def treshold_pred(lst, t=.5):
    if np.sum((lst > t).astype(int)):
        return list(lst > t).index(1)
    else:
        return -1

In [476]:
# Gaus
# w2v train 0.462 
# w2v test 0.463

# ftt train 0.455
# ftt test 0.463

# bernuli на сотые

# logistic reg
# 0.625

# catboost 
# 0.9 train
# 0.694 test

In [659]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test.astype(int), clf.predict(X_test).astype(int))

0.6886328037592303

In [701]:
preds = pd.DataFrame(clf.predict(X))

In [656]:
# #prediction with threshold
# preds = []
# for prob_lst in clf.predict_proba(X):
#     preds.append(treshold_pred(prob_lst, .4))

In [864]:
#prediction with threshold on X
X_preds = []
for prob_lst in clf.predict_proba(X):
    X_preds.append(treshold_pred(prob_lst, .36))

In [865]:
X_preds = pd.DataFrame(X_preds)

In [866]:
# X.reset_index().drop(columns='index')

In [867]:
idx = X_preds.loc[X_preds[0] >= 0].index
idx2 = X_preds.loc[X_preds[0] < 0].index

In [868]:
accuracy_score(np.array(y.reset_index()['category'].iloc[idx]).astype(int), X_preds.loc[X_preds[0] >= 0]) -\
accuracy_score(np.array(y.reset_index()['category'].iloc[idx2]).astype(int), preds.iloc[idx2])

0.4309172971258144

In [807]:
0.437

0.437

In [653]:
# pd.unique(pd.DataFrame(clf.predict(test_w2v.drop(columns='oid')))[0])

In [655]:
pd.Series(clf.predict(X)).value_counts()

4     4204
3     3264
11    3096
9     2966
1     2952
8     2934
6     2862
0     2822
7     2726
5     2717
2     2606
10    2601
dtype: int64

In [515]:
np.unique(np.array(train_w2v_min['category']))

array([1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13], dtype=object)

In [529]:
np.unique(clf.predict(X))

array([ 1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13])

In [ ]:
classes = {
    0: -1,
    'athletics': 0,
    'autosport': 1,
    'basketball': 2,
    'boardgames': 3,
    'extreme': 4,
    'football': 5,
    'hockey': 6,
    'martial_arts': 7,
    'motosport': 8,
    'tennis': 9,
    'volleyball': 10,
    'winter_sport': 11
}

In [651]:
i = 2
print(train.loc[train['category']!='esport'].iloc[i, :])
print(clf.predict(X)[i])
print(preds[i])
list(clf.predict_proba(X)[i])

# train.loc[train['category'] == 'extreme'].iloc[i,:]['text']

oid                                                 146016084
category                                             football
text        [хороший, чешский, вратарь, доминик, доминатор...
Name: 2, dtype: object
5
5


[0.001816318169761543,
 0.007304361056118438,
 0.07929002443974172,
 0.01020882506927374,
 0.00981242999427972,
 0.5341527564234991,
 0.19175641299571375,
 0.04451733066006902,
 0.005091270980528336,
 0.020149613274559774,
 0.08984342599272135,
 0.00605723094373346]

In [500]:
# answer dataset
preds = clf.predict(test_w2v.drop(columns='oid'))
answer_df = test_w2v[['oid']]

answer_df['category'] = preds

key_list = list(classes.keys())
val_list = list(classes.values())
for i in range(1, 14):
    position = val_list.index(i)
    answer_df.loc[answer_df['category']==i, 'category'] = key_list[position]

answer_df

/var/folders/n3/rc0k_vzs5ln66jll2sd1t7wh0000gn/T/ipykernel_59481/1406815035.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['category'] = preds
/var/folders/n3/rc0k_vzs5ln66jll2sd1t7wh0000gn/T/ipykernel_59481/1406815035.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df.loc[answer_df['category']==i, 'category'] = key_list[position]


,oid,category
0,749208109,basketball
1,452466036,martial_arts
2,161038103,tennis
3,663621910,boardgames
4,566255305,tennis
...,...,...
26255,169728316,boardgames
26256,279369911,volleyball
26257,600699419,boardgames
26258,560223506,martial_arts


In [501]:
answer_df.to_csv('100_for_vk.csv', index=False)